In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import gpytorch
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.stats import qmc

In [ ]:
# Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the Berger Viscous Equation parameters
c = 1.0    # Wave speed
mu = 0.1   # Viscosity coefficient
lam = 1.0  # Nonlinearity coefficient

# Define the neural network
class PINN(nn.Module):
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.net = nn.Sequential(*[
            nn.Sequential(nn.Linear(layers[i], layers[i+1]), nn.Tanh())
            for i in range(len(layers)-2)
        ] + [nn.Linear(layers[-2], layers[-1])])

    def forward(self, x, t):
        inputs = torch.cat((x, t), dim=1)
        return self.net(inputs)

def generate_collocation_points(N_f, L=1.0, T=1.0):
    # Use Latin Hypercube Sampling for better distribution
    sampler = qmc.LatinHypercube(d=2)  # 2D (x, t) space
    sample = sampler.random(N_f)  # Generate N_f samples in [0, 1]^2

    # Scale samples: x in [-L, L], t in [0, T]
    x_f = torch.tensor((sample[:, 0] * 2 - 1) * L, dtype=torch.float32).reshape(-1, 1)
    t_f = torch.tensor(sample[:, 1] * T, dtype=torch.float32).reshape(-1, 1)

    return x_f.to(device), t_f.to(device)
def compute_pde_residual(model, x, t):
    x.requires_grad = True
    t.requires_grad = True

    u = model(x, t)  # Predict u(x, t)

    u_t = torch.autograd.grad(u, t, torch.ones_like(u), create_graph=True)[0]
    u_x = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]

    # Inviscid Burgers' equation: u_t + u * u_x = 0
    f = u_t + u * u_x
    return f


# Define loss function
def loss_function(model, x_f, t_f, x_bc, t_bc, u_bc):
    f_residual = compute_pde_residual(model, x_f, t_f)
    loss_pde = torch.mean(f_residual**2)

    u_pred_bc = model(x_bc, t_bc)
    loss_bc = torch.mean((u_pred_bc - u_bc)**2)

    return loss_pde + loss_bc

# Training loop
def train(model, optimizer, x_f, t_f, valX, valT, x_bc, t_bc, u_bc, epochs=5000,threshold = 0.001):
    val_scores = []
    thresh_e = epochs
    for epoch in range(epochs):
        optimizer.zero_grad()
        loss = loss_function(model, x_f, t_f, x_bc, t_bc, u_bc)
        loss.backward()
        optimizer.step()

        loss_val = loss_function(model, valX, valT, x_bc, t_bc, u_bc)

        if loss_val.item() < threshold and thresh_e >= epochs:
            print("Threshold reach at:",epoch)
            print("Val loss:",loss_val)
            thresh_e = epoch

        if epoch % 500 == 0 or epoch == epochs-1:
            print(f"Epoch {epoch}, Loss: {loss.item()}")
            val_scores.append(loss_val.item())

    return thresh_e, val_scores

In [ ]:

def gen_points_import(model, N_f, L=1.0, T=1.0):
    x_f = (torch.rand(N_f, 1, device=device, requires_grad=True) * 2 - 1) * L  # x in [-L, L]
    t_f = torch.rand(N_f, 1, device=device, requires_grad=True) * T  # t in [0, T]
    
    if model is not None:  # Perform importance sampling
        residuals = compute_pde_residual(model, x_f, t_f).detach()
        probabilities = residuals.abs() / torch.sum(residuals.abs())
        sampled_indices = torch.multinomial(probabilities.view(-1), N_f, replacement=True)
        x_f, t_f = x_f[sampled_indices], t_f[sampled_indices]
    
    return x_f, t_f

def train_import(model, optimizer, N_f, x_f,t_f,valX ,valT ,x_bc, t_bc, u_bc, epochs=10000, resample_every=5000,threshold = 0.001):

    val_scores = []
    thresh_e = epochs
    for epoch in range(epochs):
        if epoch % resample_every == 0 and epoch >=500:
            x_f, t_f = gen_points_import(model, N_f)
        
        optimizer.zero_grad()
        loss = loss_function(model, x_f, t_f, x_bc, t_bc, u_bc)
        loss.backward()
        optimizer.step()

        loss_val = loss_function(model, valX, valT, x_bc, t_bc, u_bc)

        if loss_val.item() < threshold and thresh_e >= epochs:
            print("Threshold reach at:",epoch)
            print("Val loss:",loss_val)
            thresh_e = epoch
        if epoch % 500 == 0 or epoch == epochs-1:
            print(f"Epoch {epoch}, Loss: {loss.item()}")
            val_scores.append(loss_val.item())

    return thresh_e, val_scores

In [ ]:
# Gaussian Process Model for Importance Sampling
class ResidualGP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ResidualGP, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        # self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        self.covar_module = gpytorch.kernels.ScaleKernel(
    gpytorch.kernels.MaternKernel(nu=1.5)
)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
import torch
import gpytorch
from scipy.stats import qmc

def generate_collocation_points_with_gp(model, N_f, x_f, t_f, x_bc=None, t_bc=None, u_bc=None, 
                                        L=1.0, T=1.0, alpha=0.5, fraction_gp=0.5, residual_thresh=1e-3):
    device = x_f.device

    # === Step 1: Prepare GP training data ===
    x_train = x_f
    t_train = t_f
    xt_train = torch.cat([x_train, t_train], dim=1).detach()

    if model is not None:
        with torch.no_grad():
            u_train = model(x_train, t_train).detach().view(-1)
            xt_all = xt_train
            u_all = u_train

            if x_bc is not None and t_bc is not None and u_bc is not None:
                xt_bc = torch.cat([x_bc, t_bc], dim=1).detach()
                u_bc = u_bc.detach().view(-1)
                xt_all = torch.cat([xt_all, xt_bc], dim=0)
                u_all = torch.cat([u_all, u_bc], dim=0)

        # === Train GP ===
        likelihood = gpytorch.likelihoods.GaussianLikelihood()
        gp_model = ResidualGP(xt_all, u_all, likelihood).to(device)

        gp_model.train()
        likelihood.train()
        optimizer = torch.optim.Adam(gp_model.parameters(), lr=0.001)
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, gp_model)

        for _ in range(1000):
            optimizer.zero_grad()
            output = gp_model(xt_all)
            loss = -mll(output, u_all)
            loss.backward()
            optimizer.step()

        gp_model.eval()
        likelihood.eval()

        # === Step 2: Generate candidate points ===
        sampler = qmc.LatinHypercube(d=2)
        sample = sampler.random(10 * N_f)
        x_cand = torch.tensor(sample[:, 0] * L, dtype=torch.float32).reshape(-1, 1).to(device)
        t_cand = torch.tensor(sample[:, 1] * T, dtype=torch.float32).reshape(-1, 1).to(device)
        xt_cand = torch.cat([x_cand, t_cand], dim=1)

        # === Step 3: Sample from GP posterior ===
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            dist = gp_model(xt_cand)
            gp_samples = dist.rsample(torch.Size([1])) # shape: [1, N_cand]
            gp_sample_abs = gp_samples.squeeze(0).abs().detach()  # [N_cand]

        # === Step 4: Compute PDE residuals ===
        x_cand.requires_grad_()
        t_cand.requires_grad_()
        residual = compute_pde_residual(model, x_cand, t_cand).detach().abs().view(-1)

        # === Step 5: Normalize and combine scores ===
        residual = torch.where(residual < residual_thresh, torch.tensor(0.0, device=device), residual)

        sample_score = gp_sample_abs / (gp_sample_abs.sum() + 1e-8)
        residual_score = residual / (residual.sum() + 1e-8)

        sampling_score = alpha * sample_score + (1 - alpha) * residual_score
        sampling_score = sampling_score / (sampling_score.sum() + 1e-8)

        # === Step 6: Hybrid sampling ===
        N_gp = int(fraction_gp * N_f)
        N_rand = N_f - N_gp

        sampled_indices_gp = torch.multinomial(sampling_score, N_gp, replacement=False)
        sampled_indices_rand = torch.randint(0, len(x_cand), (N_rand,), device=device)
        sampled_indices = torch.cat([sampled_indices_gp, sampled_indices_rand], dim=0)

        x_f_new = x_cand[sampled_indices].detach().clone().requires_grad_()
        t_f_new = t_cand[sampled_indices].detach().clone().requires_grad_()
        uncertainty_top = gp_sample_abs[sampled_indices].detach().cpu()

    else:
        x_f_new = x_f.clone().detach().requires_grad_()
        t_f_new = t_f.clone().detach().requires_grad_()
        uncertainty_top = None
        gp_model = None

    return x_f_new, t_f_new, uncertainty_top, gp_model


In [ ]:
def train_GP(model, optimizer, N_f, x_f,t_f,valX, valT,x_bc, t_bc, u_bc, epochs=5000, resample_every=500,threshold = 0.001):
    thresh_e = epochs
    val_scores = []
    for epoch in range(epochs):
        # if epoch % resample_every == 0 and epoch >=500:
        #     x_f, t_f = gen_points_import(model, N_f)
        
        optimizer.zero_grad()
        loss = loss_function(model, x_f, t_f, x_bc, t_bc, u_bc)
        loss.backward()
        optimizer.step()

        loss_val = loss_function(model, valX, valT, x_bc, t_bc, u_bc)

        if loss_val.item() < threshold and thresh_e >= epochs:
            print("Threshold reach at:",epoch)
            print("Val loss:",loss_val)
            thresh_e = epoch

        if epoch % 500 == 0 or epoch == epochs-1:
            # x_f,t_f = fit_GP(x_f,t_f)
            print(f"Epoch {epoch}, Loss: {loss.item()}")
            val_scores.append(loss_val.item())
            # x_uncertain, t_uncertain,x,g = generate_collocation_points_with_gp(model,N_f,x_f, t_f) 
        if epoch % resample_every == 0 and epoch > 0 :
            x_uncertain, t_uncertain,uncertainties,gp_model = generate_collocation_points_with_gp(model,N_f,x_f, t_f,x_bc,t_bc,u_bc)

            x_f = x_uncertain
            t_f = t_uncertain

    return thresh_e, val_scores

In [ ]:
import torch
import gpytorch
from scipy.stats import qmc

def generate_collocation_points_with_gp_res(model, N_f, x_f, t_f, x_bc=None, t_bc=None, u_bc=None, 
                                            L=2.0, T=1.0, alpha=0.5, fraction_gp=0.5, residual_thresh=1e-3):

    device = x_f.device

    x_train = x_f
    t_train = t_f
    xt_train = torch.cat([x_train, t_train], dim=1).detach()

    if model is not None:
        # ✅ Compute residuals (no torch.no_grad here)
        residual_train = compute_pde_residual(model, x_train.requires_grad_(), t_train.requires_grad_()).detach().view(-1)

        xt_all = xt_train
        residual_all = residual_train

        if x_bc is not None and t_bc is not None and u_bc is not None:
            xt_bc = torch.cat([x_bc, t_bc], dim=1).detach()
            residual_bc = compute_pde_residual(model, x_bc.requires_grad_(), t_bc.requires_grad_()).detach().view(-1)

            xt_all = torch.cat([xt_all, xt_bc], dim=0)
            residual_all = torch.cat([residual_all, residual_bc], dim=0)

        # === Train GP on residuals ===
        likelihood = gpytorch.likelihoods.GaussianLikelihood()
        gp_model = ResidualGP(xt_all, residual_all, likelihood).to(device)

        gp_model.train()
        likelihood.train()
        optimizer = torch.optim.Adam(gp_model.parameters(), lr=0.001)
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, gp_model)

        for _ in range(1000):
            optimizer.zero_grad()
            output = gp_model(xt_all)
            loss = -mll(output, residual_all)
            loss.backward()
            optimizer.step()

        gp_model.eval()
        likelihood.eval()

        # === Step 2: Candidate points ===
        sampler = qmc.LatinHypercube(d=2)
        sample = sampler.random(10 * N_f)

        # x ∈ [-L/2, L/2], t ∈ [0, T]
        x_cand = torch.tensor((sample[:, 0] * 2 - 1) * (L/2), dtype=torch.float32, device=device).reshape(-1, 1)
        t_cand = torch.tensor(sample[:, 1] * T, dtype=torch.float32, device=device).reshape(-1, 1)
        xt_cand = torch.cat([x_cand, t_cand], dim=1)

        # === Step 3 (Modified): Sample from GP posterior ===
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            dist = gp_model(xt_cand)
            gp_samples = dist.rsample(torch.Size([1]))  # [1, N_cand]
            gp_sample_abs = gp_samples.squeeze(0).abs().detach()  # [N_cand]
  # [N_cand]

        # === Step 4: Compute PDE residuals at candidate points
        x_cand.requires_grad_()
        t_cand.requires_grad_()
        residual = compute_pde_residual(model, x_cand, t_cand).detach().abs().view(-1)

        # === Step 5: Normalize and threshold ===
        residual = torch.where(residual < residual_thresh, torch.tensor(0.0, device=device), residual)

        sample_score = gp_sample_abs / (gp_sample_abs.sum() + 1e-8)
        residual_score = residual / (residual.sum() + 1e-8)

        sampling_score = alpha * sample_score + (1 - alpha) * residual_score
        sampling_score = sampling_score / (sampling_score.sum() + 1e-8)

        # === Step 6: Hybrid sampling ===
        N_gp = int(fraction_gp * N_f)
        N_rand = N_f - N_gp

        sampled_indices_gp = torch.multinomial(sampling_score, N_gp, replacement=False)
        sampled_indices_rand = torch.randint(0, len(x_cand), (N_rand,), device=device)
        sampled_indices = torch.cat([sampled_indices_gp, sampled_indices_rand], dim=0)

        x_f_new = x_cand[sampled_indices].detach().clone().requires_grad_()
        t_f_new = t_cand[sampled_indices].detach().clone().requires_grad_()
        uncertainty_top = gp_sample_abs[sampled_indices].detach().cpu()

    else:
        x_f_new = x_f.clone().detach().requires_grad_()
        t_f_new = t_f.clone().detach().requires_grad_()
        uncertainty_top = None
        gp_model = None

    return x_f_new, t_f_new, uncertainty_top, gp_model


In [ ]:
def train_GP_res(model, optimizer, N_f, x_f,t_f,valX, valT,x_bc, t_bc, u_bc, epochs=5000, resample_every=500,threshold = 0.001):
    val_scores = []
    thresh_e = epochs
    for epoch in range(epochs):
        # if epoch % resample_every == 0 and epoch >=500:
        #     x_f, t_f = gen_points_import(model, N_f)
        
        optimizer.zero_grad()
        loss = loss_function(model, x_f, t_f, x_bc, t_bc, u_bc)
        loss.backward()
        optimizer.step()

        loss_val = loss_function(model, valX, valT, x_bc, t_bc, u_bc)

        if loss_val.item() < threshold and thresh_e >= epochs:
            print("Threshold reach at:",epoch)
            print("Val loss:",loss_val)
            thresh_e = epoch

        if epoch % 500 == 0 or epoch == epochs-1:
            # x_f,t_f = fit_GP(x_f,t_f)
            print(f"Epoch {epoch}, Loss: {loss.item()}")
            val_scores.append(loss_val.item())
            # x_uncertain, t_uncertain,x,g = generate_collocation_points_with_gp(model,N_f,x_f, t_f) 
        if epoch % resample_every == 0 and epoch > 0 :
            x_uncertain, t_uncertain,uncertainties,gp_model = generate_collocation_points_with_gp_res(model,N_f,x_f, t_f,x_bc,t_bc,u_bc)

            x_f = x_uncertain
            t_f = t_uncertain

    return thresh_e,val_scores

In [ ]:
def compute_pde_residual(model, x, t):
    x.requires_grad = True
    t.requires_grad = True

    u = model(x, t)  # Predict u(x, t)

    u_t = torch.autograd.grad(u, t, torch.ones_like(u), create_graph=True)[0]
    u_x = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]

    # Inviscid Burgers' equation: u_t + u * u_x = 0
    f = u_t + u * u_x
    return f

In [ ]:
def compute_pde_residual(model, x, t):
    x = x.detach().requires_grad_()
    t = t.detach().requires_grad_()

    u = model(x, t)  # Predict u(x, t)

    # Compute ∂u/∂t
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]

    # Compute ∂u/∂x
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]

    # Compute ∂²u/∂x²
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]

    # Burgers' equation residual: u_t + u * u_x - ν * u_xx
    nu = 0.01 / np.pi
    # residual = u_t + u * u_x - nu * u_xx
    residual = u_t + u * u_x 
    return residual


In [ ]:
#different initial/boundary conditions
def initial_condition(x, condition_type="sin"):
    if condition_type == "sin":
        return (-10 * torch.sin(np.pi * x.cpu())).to(device)
    elif condition_type == "gaussian":
        # return torch.exp(-10 * (x - 0.5) ** 2).to(device)
        mu=0.5
        sigma=0.1
        return torch.exp(-((x - mu)**2) / (2 * sigma**2))
    elif condition_type == "step":
        return torch.where(x < 0.5, torch.tensor(1.0, device=device), torch.tensor(0.0, device=device))
    else:
        raise ValueError("Unknown initial condition type")

def boundary_condition(x, t, boundary_type="dirichlet"):
    if boundary_type == "dirichlet":
        return torch.zeros_like(x).to(device)  # u(0,t) = 0, u(L,t) = 0
    elif boundary_type == "neumann":
        return torch.autograd.grad(model(x, t), x, torch.ones_like(x), create_graph=True)[0]
    elif boundary_type == "periodic":
        return model(x, t) - model(x + L, t)  # u(0,t) = u(L,t)
    else:
        raise ValueError("Unknown boundary condition type")


In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import gc
layers = [2, 50, 50, 50, 1]
N_f = 1000
L, T = 1.0, 1.0
N_bc = 100
epochs = 5000
threshold = 0.05

x_f, t_f = generate_collocation_points(N_f, L, T)
x_val, t_val = generate_collocation_points(10000, L, T)

initial_conditions = ["sin", "step" ,"gaussian"]
boundary_conditions = ["dirichlet", "neumann", "periodic"]


results_base = []
results_import = []
results_gauss = []
results_gauss_res = []

num_experiments = 20

for ic in initial_conditions:
    for bc in boundary_conditions:
        print(f"\n==== Running experiments for IC: {ic}, BC: {bc} ====")
        
        val_losses_base = []
        val_losses_import = []
        val_losses_gauss = []
        val_losses_gauss_res = []
        
        threshold_base_val = []
        threshold_import_val = []
        threshold_gauss_val = []
        threshold_gauss_val_res = []

        x_bc = torch.linspace(0, L, N_bc).view(-1, 1).to(device)
        t_bc = torch.zeros_like(x_bc).to(device)
        u_bc = initial_condition(x_bc, ic)
        for exp in range(num_experiments):
            
            print(f"--- Experiment {exp+1}/{num_experiments} ---")
            
            # Boundary/initial condition setup
            
            
            # ---- BASE ----
            model_base = PINN(layers).to(device)
            optimizer = optim.Adam(model_base.parameters(), lr=1e-3)
            thresh_base,base_scores = train(model_base, optimizer, x_f, t_f, x_val, t_val, x_bc, t_bc, u_bc, epochs=epochs, threshold=threshold)
            loss_val = loss_function(model_base, x_val, t_val, x_bc, t_bc, u_bc).item()
            print(loss_val)
            val_losses_base.append(loss_val)
            threshold_base_val.append(thresh_base)
            del model_base, optimizer
            gc.collect()
            torch.cuda.empty_cache()
            
            # ---- IMPORTANCE ----
            model_import = PINN(layers).to(device)
            optimizer = optim.Adam(model_import.parameters(), lr=1e-3)
            thresh_import,import_scores = train_import(model_import, optimizer, N_f, x_f, t_f, x_val, t_val, x_bc, t_bc, u_bc, epochs=epochs, resample_every=500, threshold=threshold)
            loss_val = loss_function(model_import, x_val, t_val, x_bc, t_bc, u_bc).item()
            print(loss_val)
            val_losses_import.append(loss_val)
            threshold_import_val.append(thresh_import)
            del model_import, optimizer
            gc.collect()
            torch.cuda.empty_cache()
            
            # ---- GAUSSIAN PROCESS ----
            model_GP = PINN(layers).to(device)
            optimizer = optim.Adam(model_GP.parameters(), lr=1e-3)
            thresh_gp,output_scores = train_GP(model_GP, optimizer, N_f, x_f, t_f, x_val, t_val, x_bc, t_bc, u_bc, epochs=epochs, resample_every=500, threshold=threshold)
            loss_val = loss_function(model_GP, x_val, t_val, x_bc, t_bc, u_bc).item()
            print(loss_val)
            val_losses_gauss.append(loss_val)
            threshold_gauss_val.append(thresh_gp)
            del model_GP, optimizer
            gc.collect()
            torch.cuda.empty_cache()


            # ---- GAUSSIAN PROCESS RESIDUALS----
            model_GP_res = PINN(layers).to(device)
            optimizer = optim.Adam(model_GP_res.parameters(), lr=1e-3)
            thresh_gp,res_scores = train_GP_res(model_GP_res, optimizer, N_f, x_f, t_f, x_val, t_val, x_bc, t_bc, u_bc, epochs=epochs, resample_every=500, threshold=threshold)
            loss_val = loss_function(model_GP_res, x_val, t_val, x_bc, t_bc, u_bc).item()
            print(loss_val)
            val_losses_gauss_res.append(loss_val)
            threshold_gauss_val_res.append(thresh_gp)
            del model_GP_res, optimizer
            gc.collect()
            torch.cuda.empty_cache()
        # === AVERAGE + STD ===
        avg_base, std_base = np.mean(val_losses_base), np.std(val_losses_base)
        avg_import, std_import = np.mean(val_losses_import), np.std(val_losses_import)
        avg_gauss, std_gauss = np.mean(val_losses_gauss), np.std(val_losses_gauss)
        avg_gauss_res, std_gauss_res = np.mean(val_losses_gauss_res), np.std(val_losses_gauss_res)

        avg_t_base, std_t_base = np.mean(threshold_base_val), np.std(threshold_base_val)
        avg_t_import, std_t_import = np.mean(threshold_import_val), np.std(threshold_import_val)
        avg_t_gauss, std_t_gauss = np.mean(threshold_gauss_val), np.std(threshold_gauss_val)
        avg_t_gauss_res, std_t_gauss_res = np.mean(threshold_gauss_val_res), np.std(threshold_gauss_val_res)

        print(f"Average Validation Loss (Base):   {avg_base:.6f} ± {std_base:.6f}")
        print(f"Average Validation Loss (Import): {avg_import:.6f} ± {std_import:.6f}")
        print(f"Average Validation Loss (Gauss):  {avg_gauss:.6f} ± {std_gauss:.6f}")
        print(f"Average Validation Loss (Gauss Res):  {avg_gauss_res:.6f} ± {std_gauss_res:.6f}")
        
        results_base.append([ic, bc, avg_base, std_base, avg_t_base, std_t_base])
        results_import.append([ic, bc, avg_import, std_import, avg_t_import, std_t_import])
        results_gauss.append([ic, bc, avg_gauss, std_gauss, avg_t_gauss, std_t_gauss])
        results_gauss_res.append([ic, bc, avg_gauss_res, std_gauss_res, avg_t_gauss_res, std_t_gauss_res])

        

        a = base_scores
        b = import_scores
        c = output_scores
        d = res_scores

        epochs_range = [i * 500 for i in range(11)] 
        # Plot them all on the same figure
        plt.plot(epochs_range, a, label='Base PINN loss')
        plt.plot(epochs_range,b, label='Importance Sampling PINN')
        plt.plot(epochs_range,c, label='GP Output PINN')
        plt.plot(epochs_range,d, label='GP Residual PINN')

        plt.xlabel('Epoch')
        plt.ylabel('Average Loss Over Time')
        plt.title(f'Loss Over time for IC: {ic}, BC: {bc}')
        plt.xticks(ticks=[i * 500 for i in range(11)])
        plt.legend()
        plt.grid(True)
        plt.show()
        

In [ ]:
# Example 1D arrays
        a = base_scores
        b = import_scores
        c = output_scores
        d = res_scores

        # Plot them all on the same figure
        plt.plot(a, label='base_scores')
        plt.plot(b, label='import_scores')
        plt.plot(c, label='output_scores')
        plt.plot(d, label='res_scores')

        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(f'Loss Over time for IC: {ic}, BC: {bc}')
        plt.legend()
        plt.grid(True)
        plt.show()

In [ ]:
import pandas as pd

# Convert results to DataFrames
df_base = pd.DataFrame(results_base, columns=["Initial Condition", "Boundary Condition", "Avg Validation Loss", "Std Validation Loss", "Avg Threshold","Std Threshold"])
df_import = pd.DataFrame(results_import, columns=["Initial Condition", "Boundary Condition", "Avg Validation Loss", "Std Validation Loss", "Avg Threshold","Std Threshold"])
df_gauss = pd.DataFrame(results_gauss, columns=["Initial Condition", "Boundary Condition", "Avg Validation Loss", "Std Validation Loss", "Avg Threshold","Std Threshold"])
df_gauss_res = pd.DataFrame(results_gauss_res, columns=["Initial Condition", "Boundary Condition", "Avg Validation Loss", "Std Validation Loss", "Avg Threshold","Std Threshold"])
# Save to CSV
df_base.to_csv("Results_Experiments/base_final55.csv", index=False)
df_import.to_csv("Results_Experiments/import_final55.csv", index=False)
df_gauss.to_csv("Results_Experiments/GP_final55.csv", index=False)
df_gauss_res.to_csv("Results_Experiments/GP_final_res55.csv", index=False)

In [ ]:
import pandas as pd
df_base = pd.read_csv("Results_Experiments/base_final55.csv")
df_import = pd.read_csv("Results_Experiments/import_final55.csv")
df_gauss = pd.read_csv("Results_Experiments/GP_final55.csv")
df_gauss_res = pd.read_csv("Results_Experiments/GP_final_res55.csv")


In [ ]:
df_base 

In [ ]:
df_import

In [ ]:
df_gauss

In [ ]:
df_gauss_res